# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [2]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-09-15 15:16:54.656 | INFO     | mads_datasets.base:download_data:94 - Start download...
  0%|          | 0.00/55.4M [00:00<?, ?iB/s]2025-09-15 15:16:56.703 | INFO     | mads_datasets.datatools:get_file:105 - Downloading /Users/DINGZEEFS/.cache/mads_datasets/fashionmnist/fashionmnist.pt
100%|██████████| 55.4M/55.4M [00:02<00:00, 19.8MiB/s]
2025-09-15 15:16:59.663 | INFO     | mads_datasets.base:download_data:112 - Digest of /Users/DINGZEEFS/.cache/mads_datasets/fashionmnist/fashionmnist.pt matches expected digest


We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [3]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [4]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


2025-09-15 15:19:08.108 | INFO     | mltrainer.settings:check_path:60 - Created logdir /Users/DINGZEEFS/portfolio-example/1-hypertuning-gridsearch/modellogs


We will use a very basic model: a model with three linear layers.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [6]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True, 'num_classes': 10, 'units1': 256, 'units2': 256}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [7]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [8]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2025-09-15 15:20:22.945 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs/20250915-152022
2025-09-15 15:20:27.248 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 207.46it/s]
2025-09-15 15:20:28.185 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.9242 test 0.6378 metric ['0.7769']
100%|██████████| 100/100 [00:00<00:00, 273.16it/s]
2025-09-15 15:20:28.737 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.5572 test 0.6043 metric ['0.7777']
100%|██████████| 100/100 [00:00<00:00, 271.92it/s]
2025-09-15 15:20:29.288 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.5088 test 0.5114 metric ['0.8194']
100%|██████████| 3/3 [00:01<00:00,  1.70it/s]


Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

In [9]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 256
Units: 128, 128
Units: 128, 64
Units: 64, 256
Units: 64, 128
Units: 64, 64


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [10]:
import torch

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

for unit1 in units:
    for unit2 in units:

        model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()


2025-09-15 15:21:12.597 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs/20250915-152112
2025-09-15 15:21:12.598 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:04<00:00, 209.36it/s]
2025-09-15 15:21:17.354 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.5121 test 0.4368 metric ['0.8434']
100%|██████████| 937/937 [00:04<00:00, 208.42it/s]
2025-09-15 15:21:22.217 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.3665 test 0.3936 metric ['0.8555']
100%|██████████| 937/937 [00:07<00:00, 132.14it/s]
2025-09-15 15:21:29.955 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3237 test 0.3562 metric ['0.8724']
100%|██████████| 3/3 [00:17<00:00,  5.78s/it]
2025-09-15 15:21:29.960 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs/20250915-152129
2025-09-15 15:21:29.961 | INFO     | mltrainer.trainer:__init__:68

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`

### Step 1 — Units grid (hidden-layer width): what and why
- **What are units?** Units are the number of neurons in a hidden layer (e.g., `units1`, `units2`). More units = higher model capacity.
- **Why sweep them?** Capacity affects how much structure the model can learn. Too few → underfit; too many → may overfit.
- **How we’ll compare fairly:** We’ll treat an epoch as the full dataset (`train_steps=len(train)`, `valid_steps=len(valid)`) so each epoch sees the same amount of data. We’ll keep everything else fixed and vary only the widths.
- **What to watch in TensorBoard:**
  - Training vs validation loss/accuracy.
  - A widening train–valid gap suggests overfitting; both flat and low accuracy suggests underfitting.

We’ll run a small grid: `units ∈ {64, 128, 256}` for both layers, log to `modellogs/units-grid-v1`, and make a heatmap of validation accuracy.


In [ ]:
# Reproducibility: set a global seed
import random
import numpy as np
import torch

def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [ ]:
# Helpers: run one experiment and extract final validation accuracy from logs
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Tuple, List

import torch
import torch.optim as optim
from torch import nn

# We reuse accuracy metric, trainstreamer/validstreamer from earlier cells

@dataclass
class RunResult:
    units1: int
    units2: int
    logdir: Path


def build_model(units1: int, units2: int) -> nn.Module:
    model = NeuralNetwork(num_classes=10, units1=units1, units2=units2)
    return model


def build_settings(base_logdir: str, epochs: int = 3) -> TrainerSettings:
    return TrainerSettings(
        epochs=epochs,
        metrics=[accuracy],
        logdir=base_logdir,
        train_steps=len(train),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    )


def run_single(units1: int, units2: int, base_logdir: str) -> RunResult:
    set_seed(42)
    model = build_model(units1, units2)
    settings = build_settings(base_logdir=base_logdir, epochs=3)
    loss_fn = torch.nn.CrossEntropyLoss()

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )
    trainer.loop()

    # Trainer creates a timestamped subdir; find the latest in base_logdir
    base = Path(settings.logdir)
    latest = max((p for p in base.iterdir() if p.is_dir()), key=lambda p: p.stat().st_mtime)
    return RunResult(units1=units1, units2=units2, logdir=latest)


def read_val_accuracy_from_toml(run_dir: Path) -> float:
    # settings.toml lives in the timestamped run dir; we rely on saved metrics being visible in TensorBoard.
    # If your mltrainer writes a best metric to TOML, you can parse it here. Otherwise, we’ll return NaN and use TB for visual.
    settings_file = run_dir / "settings.toml"
    if not settings_file.exists():
        return float("nan")
    try:
        import tomllib  # py311+
    except ModuleNotFoundError:
        import tomli as tomllib
    with open(settings_file, "rb") as f:
        data = tomllib.load(f)
    # Best metric may not be stored; keep placeholder
    return float("nan")


In [ ]:
# Run units1×units2 grid and build a small results table and heatmap
from itertools import product
from pathlib import Path
import math

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

units = [64, 128, 256]
base_logdir = "modellogs/units-grid-v1"
Path(base_logdir).mkdir(parents=True, exist_ok=True)

results = []
for u1, u2 in product(units, units):
    print(f"Running units1={u1}, units2={u2}")
    rr = run_single(u1, u2, base_logdir)
    val_acc = read_val_accuracy_from_toml(rr.logdir)
    results.append({"units1": u1, "units2": u2, "val_acc": val_acc, "run_dir": str(rr.logdir)})

# Results table
results_df = pd.DataFrame(results)
results_df

# Heatmap (will show NaN unless val_acc is parsed from TOML; still a template)
pivot = results_df.pivot(index="units1", columns="units2", values="val_acc")
plt.figure(figsize=(5,4))
sns.heatmap(pivot, annot=True, fmt=".3f", cmap="viridis")
plt.title("Validation accuracy by units1 × units2 (NaN if not parsed)")
plt.show()
